d #Demand Forecasting

The objective of this notebook is to illustrate how we might generate a large number of fine-grained forecasts at the store-item level in an efficient manner leveraging the distributed computational power of Databricks.  For this exercise, we will make use of an increasingly popular library for demand forecasting, [FBProphet](https://facebook.github.io/prophet/), which we will load into the notebook session associated with a cluster running Databricks 6.0 or higher:

In [2]:
# load fbprophet library
dbutils.library.installPyPI('FBProphet', version='0.5') # find latest version of fbprophet here: https://pypi.org/project/fbprophet/
dbutils.library.installPyPI('holidays','0.9.12') # this line is in response to this issue with fbprophet 0.5: https://github.com/facebook/prophet/issues/1293

dbutils.library.restartPython()

PyPI package FBProphet has been installed already. The previously-installed package is `FBProphet==0.5`. To resolve this issue detach and re-attach the notebook to create a new environment or rename the package.
PyPI package holidays has been installed already. The previously-installed package is `holidays==0.9.12`. To resolve this issue detach and re-attach the notebook to create a new environment or rename the package.

## Examine the Data

For our training dataset, we will make use of 5-years of store-item unit sales data for 50 items across 10 different stores.  This data set is publicly available as part of a past Kaggle competition and can be downloaded [here](https://www.kaggle.com/c/demand-forecasting-kernels-only/data). 

Once downloaded, we can uzip the *train.csv.zip* file and upload the decompressed CSV to */FileStore/tables/demand_forecast/train/* using the file import steps documented [here](https://docs.databricks.com/data/tables.html#create-table-ui). Please note when performing the file import, you don't need to select the *Create Table with UI* or the *Create Table in Notebook* options to complete the import process.

With the dataset accessible within Databricks, we can now explore it in preparation for modeling:

In [4]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/2015.csv,2015.csv,1806322
dbfs:/FileStore/tables/LAcasesus.csv,LAcasesus.csv,27018
dbfs:/FileStore/tables/NewyorkUSCases.csv,NewyorkUSCases.csv,24847
dbfs:/FileStore/tables/airlines.csv,airlines.csv,779982
dbfs:/FileStore/tables/coronadataset.csv,coronadataset.csv,36802504
dbfs:/FileStore/tables/coronadatasetus.csv,coronadatasetus.csv,37492352
dbfs:/FileStore/tables/coronadatasetus2.csv,coronadatasetus2.csv,26265
dbfs:/FileStore/tables/covidts.csv,covidts.csv,1169107
dbfs:/FileStore/tables/customers.csv,customers.csv,735513
dbfs:/FileStore/tables/flights.csv,flights.csv,72088113


In [5]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA, BisectingKMeans
from pyspark.sql.functions import monotonically_increasing_id

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

import re
from pyspark.sql.types import *

# structure of the training data set
train_schema = StructType([
  StructField('UID', IntegerType()),
  StructField('iso2', StringType()),
  StructField('iso3', StringType()),
  StructField('code3', IntegerType()),
  StructField('FIPS', IntegerType()),
  StructField('Admin2', StringType()),
  StructField('Lat', DoubleType()),
  StructField('CombinedKey', StringType()),
  StructField('Population', IntegerType()),
  StructField('Date', StringType()),
  StructField('Case', IntegerType()),
  StructField('Long', DoubleType()),
  StructField('Country', StringType()),
  StructField('Province', StringType())
  ])

# read the training file into a dataframe
#  orignal location:   '/FileStore/tables/demand_forecast/train/train.csv', 
train = spark.read.csv(
  '/FileStore/tables/usdeathsla.csv', 
  header=True, 
  schema=train_schema
  )

# make the dataframe queriable as a temporary view
train.createOrReplaceTempView('train')

In [6]:
train.show()
 


+--------+----+----+-----+----+-----------+-----------+--------------------+----------+----------+----+------------+-------+----------+
 UID|iso2|iso3|code3|FIPS| Admin2| Lat| CombinedKey|Population| Date|Case| Long|Country| Province|
+--------+----+----+-----+----+-----------+-----------+--------------------+----------+----------+----+------------+-------+----------+
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-01-22| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-01-23| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-01-24| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-01-25| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-01-26| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-01-27| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-01-28| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-01-29| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-01-30| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-01-31| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-02-01| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-02-02| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-02-03| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-02-04| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-02-05| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-02-06| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-02-07| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-02-08| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-02-09| 0|-118.2282411| US|California|
84006037| US| USA| 840|6037|Los Angeles|34.30828379|Los Angeles, Cali...| 10039107|2020-02-10| 0|-118.2282411| US|California|
+--------+----+----+-----+----+-----------+-----------+--------------------+----------+----------+----+------------+-------+----------+
only showing top 20 rows

When performing demand forecasting, we are often interested in general trends and seasonality.  Let's start our exploration by examing the annual trend in unit sales:

In [8]:
%sql

SELECT
  year(Date) as year, 
  sum(Case) as case
FROM train
GROUP BY year(Date)
ORDER BY year;

year,case
2020,27743


It's very clear from the data that there is a generally upward trend in total unit sales across the stores. If we had better knowledge of the markets served by these stores, we might wish to identify whether there is a maximum growth capacity we'd expect to approach over the life of our forecast.  But without that knowledge and by just quickly eyeballing this dataset, it feels safe to assume that if our goal is to make a forecast a few days, months or even a year out, we might expect continued linear growth over that time span.

Now let's examine seasonality.  If we aggregate the data around the individual months in each year, a distinct yearly seasonal pattern is observed which seems to grow in scale with overall growth in sales:

In [10]:
%sql

SELECT 
  TRUNC(Date, 'MM') as month,
  SUM(Case) as Case
FROM train
GROUP BY TRUNC(date, 'MM')
ORDER BY month;

month,Case
2020-01-01,0
2020-02-01,0
2020-03-01,279
2020-04-01,15231
2020-05-01,12233


Aggregating the data at a weekday level, a pronounced weekly seasonal pattern is observed with a peak on Sunday (weekday 0), a hard drop on Monday (weekday 1) and then a steady pickup over the week heading back to the Sunday high.  This pattern seems to be pretty stable across the five years of observations:

In [12]:
%sql

SELECT
  YEAR(Date) as year,
  CAST(DATE_FORMAT(Date, 'u') as Integer) % 7 as weekday,
  --CONCAT(DATE_FORMAT(date, 'u'), '-', DATE_FORMAT(date, 'EEEE')) as weekday,
  AVG(Case) as Case
FROM (
  SELECT 
    Date,
    SUM(Case) as Case
  FROM train
  GROUP BY Date
 ) x
GROUP BY year, CAST(DATE_FORMAT(Date, 'u') as Integer) --, CONCAT(DATE_FORMAT(date, 'u'), '-', DATE_FORMAT(date, 'EEEE'))
ORDER BY year, weekday;

year,weekday,Case
2020,0,218.86666666666667
2020,1,229.53333333333333
2020,2,245.26666666666668
2020,3,246.8125
2020,4,263.875
2020,5,279.8125
2020,6,293.125


Now that we are oriented to the basic patterns within our data, let's explore how we might build a forecast.

###Build a Forecast

Before attempting to generate forecasts for individual combinations of stores and items, it might be helpful to build a single forecast for no other reason than to orient ourselves to the use of FBProphet.

Our first step is to assemble the historical dataset on which we will train the model:

In [15]:
# query to aggregate data to date (ds) level
sql_statement = '''
  SELECT
    CAST(Date as date) as ds,
    Case as y
  FROM train
  WHERE Country='US' AND Admin2='Los Angeles'
  ORDER BY ds
  '''

# assemble dataset in Pandas dataframe
history_pd = spark.sql(sql_statement).toPandas()

# drop any missing records
history_pd = history_pd.dropna()

Now, we will import the fbprophet library, but because it can be a bit verbose when in use, we will need to fine-tune the logging settings in our environment:

In [17]:
from fbprophet import Prophet
import logging

# disable informational messages from fbprophet
logging.getLogger('py4j').setLevel(logging.ERROR)

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.

Based on our review of the data, it looks like we should set our overall growth pattern to linear and enable the evaluation of weekly and yearly seasonal patterns. We might also wish to set our seasonality mode to multiplicative as the seasonal pattern seems to grow with overall growth in sales:

In [19]:
# set model parameters
model = Prophet(
  interval_width=0.95,
  growth='linear',
  daily_seasonality=False,
  weekly_seasonality=True,
  yearly_seasonality=True,
  seasonality_mode='multiplicative'
  )

# fit the model to historical data
model.fit(history_pd)

Out[5]: <fbprophet.forecaster.Prophet at 0x7fea813ab278>

Now that we have a trained model, let's use it to build a 90-day forecast:

In [21]:
# define a dataset including both historical dates & 90-days beyond the last available date
future_pd = model.make_future_dataframe(
  periods=90, 
  freq='d', 
  include_history=True
  )

# predict over the dataset
forecast_pd = model.predict(future_pd)

display(forecast_pd)

ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
2020-01-22T00:00:00.000+0000,-11.515376219222777,-9.984148005643181,11.875943981212414,-11.515376219222777,-11.515376219222777,-1.099989402987598,-1.099989402987598,-1.099989402987598,-0.00431225077185881,-0.00431225077185881,-0.00431225077185881,-1.0956771522157394,-1.0956771522157394,-1.0956771522157394,0.0,0.0,0.0,1.1514155933376695
2020-01-23T00:00:00.000+0000,-11.163349889173185,-9.766187862034268,11.08617218959919,-11.163349889173185,-11.163349889173185,-1.0516005115714493,-1.0516005115714493,-1.0516005115714493,0.033121369323355604,0.033121369323355604,0.033121369323355604,-1.084721880894805,-1.084721880894805,-1.084721880894805,0.0,0.0,0.0,0.5760345651324181
2020-01-24T00:00:00.000+0000,-10.811323559123595,-11.005496592419181,10.420681364480568,-10.811323559123595,-10.811323559123595,-1.006629047950226,-1.006629047950226,-1.006629047950226,0.06549296497723146,0.06549296497723146,0.06549296497723146,-1.0721220129274573,-1.0721220129274573,-1.0721220129274573,0.0,0.0,0.0,0.07166878227883842
2020-01-25T00:00:00.000+0000,-10.459297229074005,-11.235860800125486,10.479289297605442,-10.459297229074005,-10.459297229074005,-0.9909660547558717,-0.9909660547558717,-0.9909660547558717,0.06769679141397514,0.06769679141397514,0.06769679141397514,-1.0586628461698468,-1.0586628461698468,-1.0586628461698468,0.0,0.0,0.0,-0.09448871845951701
2020-01-26T00:00:00.000+0000,-10.107270816268914,-9.638000797669008,11.101045663044717,-10.107270816268914,-10.107270816268914,-1.0716721227983863,-1.0716721227983863,-1.0716721227983863,-0.026572083991053305,-0.026572083991053305,-0.026572083991053305,-1.045100038807333,-1.045100038807333,-1.045100038807333,0.0,0.0,0.0,0.7244095551001718
2020-01-27T00:00:00.000+0000,-9.755244403463822,-8.959414797979413,12.115846064345593,-9.755244403463822,-9.755244403463822,-1.1189714919521134,-1.1189714919521134,-1.1189714919521134,-0.08683314123095293,-0.08683314123095293,-0.08683314123095293,-1.0321383507211606,-1.0321383507211606,-1.0321383507211606,0.0,0.0,0.0,1.1605959810375956
2020-01-28T00:00:00.000+0000,-9.403217990658732,-9.519978839478412,11.63103287370129,-9.403217990658732,-9.403217990658732,-1.0690061308284906,-1.0690061308284906,-1.0690061308284906,-0.04859364972047617,-0.04859364972047617,-0.04859364972047617,-1.0204124811080146,-1.0204124811080146,-1.0204124811080146,0.0,0.0,0.0,0.648879690872213
2020-01-29T00:00:00.000+0000,-9.05119157785364,-10.583833392632812,9.995410535383884,-9.05119157785364,-9.05119157785364,-1.0147827951916093,-1.0147827951916093,-1.0147827951916093,-0.0043122507718547955,-0.0043122507718547955,-0.0043122507718547955,-1.0104705444197546,-1.0104705444197546,-1.0104705444197546,0.0,0.0,0.0,0.13380191133542937
2020-01-30T00:00:00.000+0000,-8.699165164689925,-11.097180080180866,10.759384379981615,-8.699165164689925,-8.699165164689925,-0.9696392711914412,-0.9696392711914412,-0.9696392711914412,0.03312136932337807,0.03312136932337807,0.03312136932337807,-1.0027606405148193,-1.0027606405148193,-1.0027606405148193,0.0,0.0,0.0,-0.2641129944260124
2020-01-31T00:00:00.000+0000,-8.347138751526211,-10.959408305187935,10.030298266349506,-8.347138751526211,-8.347138751526211,-0.9321279104882594,-0.9321279104882594,-0.9321279104882594,0.06549296497712884,0.06549296497712884,0.06549296497712884,-0.9976208754653881,-0.9976208754653881,-0.9976208754653881,0.0,0.0,0.0,-0.5665377485105059


How did our model perform? Here we can see the general and seasonal trends in our model presented as graphs:

In [23]:
trends_fig = model.plot_components(forecast_pd)
display(trends_fig)

And here, we can see how our actual and predicted data line up as well as a forecast for the future, though we will limit our graph to the last year of historical data just to keep it readable:

In [25]:
predict_fig = model.plot( forecast_pd, xlabel='Date', ylabel='Case')

# adjust figure to display dates from last year + the 90 day forecast
xlim = predict_fig.axes[0].get_xlim()
new_xlim = ( xlim[1]-(245.0), xlim[1]-45.0)
predict_fig.axes[0].set_xlim(new_xlim)

display(predict_fig)

**NOTE** This visualization is a bit busy. Bartosz Mikulski provides [an excellent breakdown](https://www.mikulskibartosz.name/prophet-plot-explained/) of it that is well worth checking out.  In a nutshell, the black dots represent our actuals with the darker blue line representing our predictions and the lighter blue band representing our (95%) uncertainty interval.

Visual inspection is useful, but a better way to evaulate the forecast is to calculate Mean Absolute Error, Mean Squared Error and Root Mean Squared Error values for the predicted relative to the actual values in our set:

In [28]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from datetime import date

# get historical actuals & predictions for comparison
actuals_pd = history_pd[ history_pd['ds'] < date(2020, 5, 6) ]['y']
predicted_pd = forecast_pd[ forecast_pd['ds'] < date(2020, 5, 6) ]['yhat']

# calculate evaluation metrics
mae = mean_absolute_error(actuals_pd, predicted_pd)
mse = mean_squared_error(actuals_pd, predicted_pd)
rmse = sqrt(mse)

# print metrics to the screen
print( '\n'.join(['MAE: {0}', 'MSE: {1}', 'RMSE: {2}']).format(mae, mse, rmse) )

/local_disk0/tmp/1589214853828-0/PythonShell.py:7: FutureWarning: Comparing Series of datetimes with 'datetime.date'. Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
 import pickle
MAE: 2.791683770557852
MSE: 29.630826341588115
RMSE: 5.4434204634207815

FBProphet provides [additional means](https://facebook.github.io/prophet/docs/diagnostics.html) for evaluating how your forecasts hold up over time. You're strongly encouraged to consider using these and those additional techniques when building your forecast models but we'll skip this here to focus on the scaling challenge.

###Scaling Model Training & Forecasting

With the mechanics under our belt, let's now tackle our original goal of building numerous, fine-grain models & forecasts for individual store and item combinations.  We will start by assembling sales data at the store-item-date level of granularity:

**NOTE**: The data in this data set should already be aggregated at this level of granularity but we are explicitly aggregating to ensure we have the expected data structure.

In [31]:
sql_statement = '''
  SELECT
    Province,
    Admin2,
    CAST(Date as Date) as ds,
    SUM(Case) as y
  FROM train
  GROUP BY Province, Admin2, ds
  ORDER BY Province, Admin2, ds
  '''

store_item_history = (
  spark
    .sql( sql_statement )
    .repartition(sc.defaultParallelism, ['Province', 'Admin2'])
  ).cache()

With our data aggregated at the store-item-date level, we need to consider how we will pass our data to FBProphet. If our goal is to build a model for each store and item combination, we will need to pass in a store-item subset from the dataset we just assembled, train a model on that subset, and receive a store-item forecast back. We'd expect that forecast to be returned as a dataset with a structure like this where we retain the store and item identifiers for which the forecast was assembled and we limit the output to just the relevant subset of fields generated by the Prophet model:

In [33]:
from pyspark.sql.types import *

result_schema =StructType([
  StructField('ds',DateType()),
  StructField('Province',StringType()),
  StructField('Admin2',StringType()),
  StructField('y',FloatType()),
  StructField('yhat',FloatType()),
  StructField('yhat_upper',FloatType()),
  StructField('yhat_lower',FloatType())
  ])

To train the model and generate a forecast we will leverage a Pandas user-defined function (UDF).  We will define this function to receive a subset of data organized around a store and item combination.  It will return a forecast in the format identified in the previous cell:

In [35]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf( result_schema, PandasUDFType.GROUPED_MAP )
def forecast_store_item( history_pd ):
  
  # TRAIN MODEL AS BEFORE
  # --------------------------------------
  # remove missing values (more likely at day-store-item level)
  history_pd = history_pd.dropna()
  
  # configure the model
  model = Prophet(
    interval_width=0.95,
    growth='linear',
    daily_seasonality=False,
    weekly_seasonality=True,
    yearly_seasonality=True,
    seasonality_mode='multiplicative'
    )
  
  # train the model
  model.fit( history_pd )
  # --------------------------------------
  
  # BUILD FORECAST AS BEFORE
  # --------------------------------------
  # make predictions
  future_pd = model.make_future_dataframe(
    periods=90, 
    freq='d', 
    include_history=True
    )
  forecast_pd = model.predict( future_pd )  
  # --------------------------------------
  
  # ASSEMBLE EXPECTED RESULT SET
  # --------------------------------------
  # get relevant fields from forecast
  f_pd = forecast_pd[ ['ds','yhat', 'yhat_upper', 'yhat_lower'] ].set_index('ds')
  
  # get relevant fields from history
  h_pd = history_pd[['ds','Province','Admin2','y']].set_index('ds')
  
  # join history and forecast
  results_pd = f_pd.join( h_pd, how='left' )
  results_pd.reset_index(level=0, inplace=True)
  
  # get store & item from incoming data set
  results_pd['Province'] = history_pd['Province'].iloc[0]
  results_pd['Admin2'] = history_pd['Admin2'].iloc[0]
  # --------------------------------------
  
  # return expected dataset
  return results_pd[ ['ds', 'Province', 'Admin2', 'y', 'yhat', 'yhat_upper', 'yhat_lower'] ]  

There's a lot taking place within our UDF, but if you compare the first two blocks of code within which the model is being trained and a forecast is being built to the cells in the previous portion of this notebook, you'll see the code is pretty much the same as before. It's only in the assembly of the required result set that truly new code is being introduced and it consists of fairly standard Pandas dataframe manipulations.

Now let's call our UDF to build our forecasts.  We do this by grouping our historical dataset around store and item.  We then apply our UDF to each group and tack on today's date as our *training_date* for data management purposes:

In [38]:
from pyspark.sql.functions import current_date

results = (
  store_item_history
    .groupBy('Province', 'Admin2')
    .apply(forecast_store_item)
    .withColumn('training_date', current_date() )
    )

results.createOrReplaceTempView('new_forecasts4')

In [39]:
%sql
Select * from new_forecasts4;

ds,Province,Admin2,y,yhat,yhat_upper,yhat_lower,training_date
2020-01-22,California,Los Angeles,0.0,1.1514156,12.210131,-8.970421,2020-05-11
2020-01-23,California,Los Angeles,0.0,0.57603455,10.533847,-10.313133,2020-05-11
2020-01-24,California,Los Angeles,0.0,0.07166878,10.121113,-10.899499,2020-05-11
2020-01-25,California,Los Angeles,0.0,-0.09448872,11.0351715,-10.206735,2020-05-11
2020-01-26,California,Los Angeles,0.0,0.7244096,10.803239,-9.646,2020-05-11
2020-01-27,California,Los Angeles,0.0,1.160596,12.110899,-9.923617,2020-05-11
2020-01-28,California,Los Angeles,0.0,0.6488797,11.607195,-9.539299,2020-05-11
2020-01-29,California,Los Angeles,0.0,0.1338019,10.916021,-10.071932,2020-05-11
2020-01-30,California,Los Angeles,0.0,-0.264113,10.733223,-10.627228,2020-05-11
2020-01-31,California,Los Angeles,0.0,-0.56653774,9.628358,-10.838116,2020-05-11


We we are likely wanting to report on our forecasts, so let's save them to a queriable table structure:

In [41]:
%sql
-- create forecast table
create table if not exists forecasts4 (
  date date,
  Province string,
  Admin2 string,
  case float,
  case_predicted float,
  case_predicted_upper float,
  case_predicted_lower float,
  training_date date
  )
using delta
partitioned by (training_date);



In [42]:
%sql
select 
  ds as date,
  Province,
  Admin2,
  y as case,
  yhat as case_predicted,
  yhat_upper as case_predicted_upper,
  yhat_lower as case_predicted_lower,
  training_date
from new_forecasts4;

date,Province,Admin2,case,case_predicted,case_predicted_upper,case_predicted_lower,training_date
2020-01-22,California,Los Angeles,0.0,1.1514156,11.650987,-9.86011,2020-05-11
2020-01-23,California,Los Angeles,0.0,0.57603455,11.673495,-10.18693,2020-05-11
2020-01-24,California,Los Angeles,0.0,0.07166878,10.797862,-11.395988,2020-05-11
2020-01-25,California,Los Angeles,0.0,-0.09448872,10.652769,-9.812909,2020-05-11
2020-01-26,California,Los Angeles,0.0,0.7244096,11.767069,-9.747045,2020-05-11
2020-01-27,California,Los Angeles,0.0,1.160596,11.272841,-9.342418,2020-05-11
2020-01-28,California,Los Angeles,0.0,0.6488797,11.121194,-10.120262,2020-05-11
2020-01-29,California,Los Angeles,0.0,0.1338019,11.491744,-10.596238,2020-05-11
2020-01-30,California,Los Angeles,0.0,-0.264113,11.147969,-11.136167,2020-05-11
2020-01-31,California,Los Angeles,0.0,-0.56653774,10.09772,-11.192444,2020-05-11


In [43]:
 %sql
-- load data to it
insert into forecasts4
select 
  ds as date,
  Province,
  Admin2,
 y as case,
  yhat as case_predicted,
  yhat_upper as case_predicted_upper,
  yhat_lower as case_predicted_lower,
  training_date
from new_forecasts4;

In [44]:
%sql
Select * from forecasts4;

date,Province,Admin2,case,case_predicted,case_predicted_upper,case_predicted_lower,training_date
2020-01-22,California,Los Angeles,0.0,1.1514156,11.341393,-8.929159,2020-05-11
2020-01-23,California,Los Angeles,0.0,0.57603455,11.006893,-10.344334,2020-05-11
2020-01-24,California,Los Angeles,0.0,0.07166878,10.609832,-10.017914,2020-05-11
2020-01-25,California,Los Angeles,0.0,-0.09448872,9.922335,-10.655466,2020-05-11
2020-01-26,California,Los Angeles,0.0,0.7244096,11.061055,-9.848713,2020-05-11
2020-01-27,California,Los Angeles,0.0,1.160596,12.808715,-9.201336,2020-05-11
2020-01-28,California,Los Angeles,0.0,0.6488797,10.93278,-9.421794,2020-05-11
2020-01-29,California,Los Angeles,0.0,0.1338019,10.450925,-10.313462,2020-05-11
2020-01-30,California,Los Angeles,0.0,-0.264113,10.012919,-11.224437,2020-05-11
2020-01-31,California,Los Angeles,0.0,-0.56653774,9.970888,-11.24085,2020-05-11


But how good (or bad) is each forecast?  Using the UDF technique, we can generate evaluation metrics for each store-item forecast as follows:

In [46]:
import pandas as pd

# schema of expected result set
eval_schema =StructType([
  StructField('training_date', DateType()),
  StructField('Province', StringType()),
  StructField('Admin2', StringType()),
  StructField('mae', FloatType()),
  StructField('mse', FloatType()),
  StructField('rmse', FloatType())
  ])

# define udf to calculate metrics
@pandas_udf( eval_schema, PandasUDFType.GROUPED_MAP )
def evaluate_forecast( evaluation_pd ):
  
  # get store & item in incoming data set
  training_date = evaluation_pd['training_date'].iloc[0]
  Province = evaluation_pd['Province'].iloc[0]
  Admin2 = evaluation_pd['Admin2'].iloc[0]
  
  # calulate evaluation metrics
  mae = mean_absolute_error( evaluation_pd['y'], evaluation_pd['yhat'] )
  mse = mean_squared_error( evaluation_pd['y'], evaluation_pd['yhat'] )
  rmse = sqrt( mse )
  
  # assemble result set
  results = {'training_date':[training_date], 'Province':[Province], 'Admin2':[Admin2], 'mae':[mae], 'mse':[mse], 'rmse':[rmse]}
  return pd.DataFrame.from_dict( results )

# calculate metrics
results = (
  spark
    .table('new_forecasts4')
    .filter('ds < \'2020-05-06\'') # limit evaluation to periods where we have historical data
    .select('training_date', 'Province', 'Admin2', 'y', 'yhat')
    .groupBy('training_date', 'Province', 'Admin2')
    .apply(evaluate_forecast)
    )
results.createOrReplaceTempView('new_forecast_evals4')



Once again, we will likely want to report the metrics for each forecast, so we persist these to a queriable table:

In [48]:
%sql

create table if not exists forecast_evals4 (
  Province string,
  Admin2 string,
  mae float,
  mse float,
  rmse float,
  training_date date
  )
using delta
partitioned by (training_date);

insert into forecast_evals4
select
  Province,
  Admin2,
  mae,
  mse,
  rmse,
  training_date
from new_forecast_evals4;

In [49]:
%sql
Select * from forecasts4;

date,Province,Admin2,case,case_predicted,case_predicted_upper,case_predicted_lower,training_date
2020-01-22,California,Los Angeles,0.0,1.1514156,11.341393,-8.929159,2020-05-11
2020-01-23,California,Los Angeles,0.0,0.57603455,11.006893,-10.344334,2020-05-11
2020-01-24,California,Los Angeles,0.0,0.07166878,10.609832,-10.017914,2020-05-11
2020-01-25,California,Los Angeles,0.0,-0.09448872,9.922335,-10.655466,2020-05-11
2020-01-26,California,Los Angeles,0.0,0.7244096,11.061055,-9.848713,2020-05-11
2020-01-27,California,Los Angeles,0.0,1.160596,12.808715,-9.201336,2020-05-11
2020-01-28,California,Los Angeles,0.0,0.6488797,10.93278,-9.421794,2020-05-11
2020-01-29,California,Los Angeles,0.0,0.1338019,10.450925,-10.313462,2020-05-11
2020-01-30,California,Los Angeles,0.0,-0.264113,10.012919,-11.224437,2020-05-11
2020-01-31,California,Los Angeles,0.0,-0.56653774,9.970888,-11.24085,2020-05-11


We now have constructed a forecast for each store-item combination and generated basic evaluation metrics for each.  To see this forecast data, we can issue a simple query (limited here to product 1 across stores 1 through 10):

In [51]:
%sql

SELECT
  Province,
  date,
  Admin2,
  case_predicted,
  case_predicted_upper,
  case_predicted_lower
FROM forecasts4 a
WHERE Admin2 = 'Los Angeles' AND
      --store IN (1, 2, 3, 4, 5) AND
      date >= '2020-05-01' AND
      training_date=current_date()
ORDER BY Province

Province,date,Admin2,case_predicted,case_predicted_upper,case_predicted_lower
California,2020-05-01,Los Angeles,1169.3247,1179.9493,1159.7285
California,2020-05-02,Los Angeles,1216.002,1227.1147,1205.5752
California,2020-05-03,Los Angeles,1231.2102,1242.0618,1219.9501
California,2020-05-04,Los Angeles,1255.0796,1266.4187,1244.3198
California,2020-05-05,Los Angeles,1310.7618,1321.5514,1299.752
California,2020-05-06,Los Angeles,1368.22,1379.2545,1357.943
California,2020-05-07,Los Angeles,1422.4707,1433.278,1412.5941
California,2020-05-08,Los Angeles,1473.454,1484.0652,1462.3099
California,2020-05-09,Los Angeles,1510.6205,1520.9049,1499.5089
California,2020-05-10,Los Angeles,1505.6654,1516.7045,1494.4482


And for each of these, we can retrieve a measure of help us assess the reliability of each forecast:

In [53]:
%sql

SELECT
  Province,
  mae,
  mse,
  rmse
FROM forecast_evals4 a
WHERE Admin2 = 'Los Angeles' AND
      training_date=current_date()
ORDER BY Province

Province,mae,mse,rmse
California,2.7916834,29.630835,5.4434214
